In [1]:
import sys
import numpy as np
from versioned_hdf5.subchunk_map import as_subchunk_map
from versioned_hdf5.staged_changes import (
    ChangesPlan,
    GetItemPlan,
    ResizePlan,
    SetItemPlan, 
    _fmt_fancy_index,
)

np.random.seed(0)

print(sys.modules["versioned_hdf5.hyperspace"])
print(sys.modules["versioned_hdf5.staged_changes"])
print(sys.modules["versioned_hdf5.subchunk_map"])

<module 'versioned_hdf5.hyperspace' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/hyperspace.cpython-312-x86_64-linux-gnu.so'>
<module 'versioned_hdf5.staged_changes' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/staged_changes.cpython-312-x86_64-linux-gnu.so'>
<module 'versioned_hdf5.subchunk_map' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/subchunk_map.cpython-312-x86_64-linux-gnu.so'>


In [2]:
# Very important flag. Read versioned_hdf5/wrappers.py
USE_VIRTUAL_GETITEM = True

# GetItemPlan benchmarks

In [3]:
def bench_getitem_plan(mode, chunk_shape, modified_perc = None):
    if not USE_VIRTUAL_GETITEM and mode != "all change":
        print(f"{mode} is not a valid use case when USE_VIRTUAL_GETITEM is False")
        return

    if chunk_shape == "square":
        chunk_size = (1000, 1000)
        nchunks = (100, 100)
    elif chunk_shape == "flat":
        chunk_size = (10, 100_000)
        nchunks = (10_000, 1)
    else:
        raise ValueError(chunk_shape)

    shape = (100_000, 100_000)
    assert all(n*c == s for n, c, s in zip(nchunks, chunk_size, shape))

    chunk_states = np.zeros(nchunks, dtype=np.intp)

    idx = ()

    if mode == "no change":
        pass

    elif mode == "one change":
        chunk_states.flat[np.random.randint(chunk_states.size)] = 1

    elif mode == "all change":
        chunk_states.flat[:] = np.arange(1, chunk_states.size + 1)

    elif mode == "dense":
        nmodified = int(chunk_states.size * modified_perc)

        if chunk_shape == "square":
            nmr = int(np.sqrt(nmodified))
            nmc = nmodified // nmr
            startr = np.random.randint(0, nchunks[0] - nmr)
            startc = np.random.randint(0, nchunks[1] - nmc)
            chunk_states[startr:startr + nmr, startc:startc + nmc] = np.arange(1, nmr * nmc + 1).reshape(nmr, nmc)
        else:
            start = np.random.randint(0, chunk_states.size - nmodified)
            chunk_states[start:start + nmodified, :] = np.arange(1, nmodified + 1).reshape(-1, 1)

    elif mode == "sparse":
        nmodified = int(chunk_states.size * modified_perc)

        chunk_states.flat[
            np.random.choice(np.arange(chunk_states.size), nmodified, replace=False)
        ] = np.arange(1, nmodified + 1)

    else:
        raise ValueError(mode)

    args = (idx, chunk_size, shape, chunk_states)

    if mode in ("dense", "sparse"):
        print(f"{mode} {modified_perc * 100:.0f}%, {chunk_shape}")
    else:
        print(f"{mode}, {chunk_shape}")
    print(GetItemPlan(*args).head)
    %timeit GetItemPlan(*args)
    print()


Performance is impacted by the number of modified chunks.


All benchmarks run on a dataset 80 GB in size (assuming double or int64 data), 10,000 chunks, 8 MB per chunk and plan the execution of reading the whole dataset item (a[:, :]).

#### Legend
Shape of the chunks:
- **square**: 1000x1000 points (8MB) square chunks, 100 chunks per axis
- **flat**: a single chunk of 8MB spanning the columns back to back. Typical e.g after a conversion from Pandas or PyArrow.

What kind of `__setitem__` calls happened before this `__getitem__`?
- **One change:** a single chunk randomly in the center has been modified (`a[r, c] = 1`). Need to read up 9,999 chunks from disk.
- **Dense:** a monolithic block of chunks in the middle of the array has been modified (`a[r0:r1, c0:c1] = 1`). Tested for 5% of the total surface and for 80%; respectively 9500 and  and 2000 chunks need to be loaded from disk.
- **Sparse:** individual chunks randomly selected have been modified. Tested for 5% of the total surface and for 80%.

The number of calls to `h5py.Dataset.__getitem__` is highlighted by the plans as _direct reads_.

In [4]:
bench_getitem_plan("no change", "square")
bench_getitem_plan("one change", "square")
bench_getitem_plan("dense", "square", 0.05)
bench_getitem_plan("sparse", "square", 0.05)
bench_getitem_plan("dense", "square", 0.8)
bench_getitem_plan("sparse", "square", 0.8)
bench_getitem_plan("all change", "square")

bench_getitem_plan("no change", "flat")
bench_getitem_plan("one change", "flat")
bench_getitem_plan("dense", "flat", 0.05)
bench_getitem_plan("sparse", "flat", 0.05)
bench_getitem_plan("dense", "flat", 0.8)
bench_getitem_plan("sparse", "flat", 0.8)
bench_getitem_plan("all change", "flat")

no change, square
GetItemPlan<shape=(100000, 100000), 0 modified chunks, 0 full chunks, 1 direct reads>
55.1 μs ± 1.18 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

one change, square
GetItemPlan<shape=(100000, 100000), 1 modified chunks, 0 full chunks, 4 direct reads>
62.3 μs ± 2.14 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

dense 5%, square
GetItemPlan<shape=(100000, 100000), 484 modified chunks, 0 full chunks, 46 direct reads>
404 μs ± 369 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

sparse 5%, square
GetItemPlan<shape=(100000, 100000), 500 modified chunks, 0 full chunks, 560 direct reads>
673 μs ± 1.11 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

dense 80%, square
GetItemPlan<shape=(100000, 100000), 7921 modified chunks, 0 full chunks, 180 direct reads>
5.6 ms ± 34.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)

sparse 80%, square
GetItemPlan<shape=(100000, 100000), 8000 modified chunks, 0 full chunks, 1629 

# as_subchunk_map legacy benchmarks
Same dataset and chunks shapes as in the GetItemPlan and SetItemPlan benchmarks.
Performance of as_subchunk_map() is unaffected by how many chunks are loaded from disk or where the changes are.

In `__getitem__`, this will trigger

- **one change:** 9,999 calls to `h5py.Dataset.__getitem__`
- **5%**: 9,500 calls
- **80%**: 2,000 calls

In `__setitem__`, this will always load all the impacted chunks that weren't already in memory.

In [5]:
%%timeit
# square
_ = list(as_subchunk_map(
    chunk_size=(1000, 1000),
    idx=(),
    shape=(100_000, 100_000),
))

30.3 ms ± 102 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
# flat
_ = list(as_subchunk_map(
    chunk_size=(10, 100_000),
    idx=(),
    shape=(100_000, 100_000),
))

40.2 ms ± 103 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# More plans benchmarks
The benchmarks below mirror those in wrapper_benchmark.ipynb

In [7]:
shape = (20_000, 20_000)
chunk_size = (100, 100)
nchunks = tuple(s // c + (s % c > 0) for s, c in zip(shape, chunk_size))
if USE_VIRTUAL_GETITEM:
    chunk_states = np.zeros(nchunks, dtype=np.intp)
else:
    chunk_states = np.arange(1, np.prod(nchunks) + 1, dtype=np.intp).reshape(nchunks)

In [8]:
# setitem of whole chunks not in memory
idx = slice(chunk_size[0], None, None)
splan = SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
print(splan.head)
%timeit SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
%timeit list(as_subchunk_map(chunk_size, idx, shape))

SetItemPlan<shape=(19900, 20000), 0 loads from base into 0 chunks, 0 appends of empty chunks, 0 appends of full chunks, 39800 appends from __setitem__ value, 0 replaces with empty chunks, 0 replaces from __setitem__ value, 0 updates>
56.5 ms ± 124 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
123 ms ± 580 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
# setitem of parts of chunks (target all chunks)
idx = tuple(slice(None, None, c) for c in chunk_size)
splan = SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
print(splan.head)
%timeit SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
%timeit list(as_subchunk_map(chunk_size, idx, shape))

SetItemPlan<shape=(200, 200), 1 loads from base into 40000 chunks, 0 appends of empty chunks, 0 appends of full chunks, 0 appends from __setitem__ value, 0 replaces with empty chunks, 0 replaces from __setitem__ value, 40000 updates>
24.4 ms ± 604 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
126 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
# setitem of parts of chunks (target every other chunk)
idx = (slice(None, None, chunk_size[0] * 2), slice(None, None, chunk_size[1]))
splan = SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
print(splan.head)
%timeit SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
%timeit list(as_subchunk_map(chunk_size, idx, shape))

SetItemPlan<shape=(100, 200), 100 loads from base into 20000 chunks, 0 appends of empty chunks, 0 appends of full chunks, 0 appends from __setitem__ value, 0 replaces with empty chunks, 0 replaces from __setitem__ value, 20000 updates>
15.2 ms ± 428 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
62.4 ms ± 493 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
# setitem of parts of chunks (target every other chunk on inner axis)
idx = (slice(None, None, chunk_size[0]), slice(None, None, chunk_size[1] * 2))
splan = SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
print(splan.head)
%timeit SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
%timeit list(as_subchunk_map(chunk_size, idx, shape))

SetItemPlan<shape=(200, 100), 100 loads from base into 20000 chunks, 0 appends of empty chunks, 0 appends of full chunks, 0 appends from __setitem__ value, 0 replaces with empty chunks, 0 replaces from __setitem__ value, 20000 updates>
12.8 ms ± 33.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
63.3 ms ± 775 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
# setitem of parts of chunks (checkers pattern)
idx = (slice(None, None, chunk_size[0] * 2), slice(None, None, chunk_size[1] * 2))
splan = SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
print(splan.head)
%timeit SetItemPlan(idx, chunk_size, shape, chunk_states, 1)
%timeit list(as_subchunk_map(chunk_size, idx, shape))

SetItemPlan<shape=(100, 100), 10000 loads from base into 10000 chunks, 0 appends of empty chunks, 0 appends of full chunks, 0 appends from __setitem__ value, 0 replaces with empty chunks, 0 replaces from __setitem__ value, 10000 updates>
31.8 ms ± 918 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.4 ms ± 249 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
# commit a fully modified array
splan = SetItemPlan((), chunk_size, shape, chunk_states, 1)
cplan = ChangesPlan(chunk_size, shape, shape, splan.chunk_states, load_base=False)
print(cplan.head)
%timeit ChangesPlan(chunk_size, shape, shape, splan.chunk_states, load_base=False)

ChangesPlan<0 deleted chunks, 40000 chunks in memory, 0 loads from base into 0 chunks>
19 ms ± 374 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
# getitem after a single chunk within the selection has been changed
splan = SetItemPlan((10000, 10000), chunk_size, shape, chunk_states, 1)
idx = ()
gplan = GetItemPlan((), chunk_size, shape, splan.chunk_states)
print(gplan.head)
%timeit GetItemPlan((), chunk_size, shape, splan.chunk_states)
%timeit list(as_subchunk_map(chunk_size, idx, shape))

GetItemPlan<shape=(20000, 20000), 1 modified chunks, 0 full chunks, 4 direct reads>
124 μs ± 1.37 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
124 ms ± 1.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
# getitem after a single chunk outside of the selection has been changed
splan = SetItemPlan((0, 0), chunk_size, shape, chunk_states, 1)
idx = slice(chunk_size[0], None, None)
gplan = GetItemPlan(idx, chunk_size, shape, splan.chunk_states)
print(gplan.head)
%timeit GetItemPlan(idx, chunk_size, shape, splan.chunk_states)
%timeit list(as_subchunk_map(chunk_size, idx, shape))

GetItemPlan<shape=(19900, 20000), 0 modified chunks, 0 full chunks, 1 direct reads>
235 μs ± 4.05 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
126 ms ± 976 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
# resize to add 1 chunk
new_shape = (shape[0] + 1, shape[1])
rplan = ResizePlan(chunk_size, shape, new_shape, chunk_states, 1)
print(rplan.head)
%timeit ResizePlan(chunk_size, shape, new_shape, chunk_states, 1)
%timeit list(as_subchunk_map(chunk_size, (), new_shape))

ResizePlan<0 loads from base into 0 chunks, 0 updates, 0 deletes>
43.1 μs ± 51 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
125 ms ± 945 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
